## Using C++

Author: Tom Wang @[tomwym](https://github.com/tomwym)

Hello! This document will go over the basics of building a DLL with C++ that can be used in Unity C#.

Unity works within the Microsoft .NET framework and uses C# as the scripting language. This means that a shared library (DLL) built within the same framework can be used instead of compiled native C# code. Why might we want to do this? C# is easy to write and understand, but at the end of the day it is much slower than binaries built using C++. You probably know all this already, if you're reading this you maybe feel more comfortable using C++ than C#!

What is a C++ library? Essentially, it is a package containing compiled code. Normally we write programs with a `main` function, but in a library we omit the `main`. Instead, we have a collection of functions which can be called.


#### Building on Windows

Unfortunately this document will only cover the Windows variants. If you're working on this project this should be fine. Have a version of Visual Studio installed, plus the appropriate packages used to build C++. The packages that will be useful this project are:  
- .NET desktop development
- Desktop development with C++
- Universal Windows Platform development

Once these tools are installed, create a new VS project. Here you can select Dynamic-Link Library (DLL) and name it accordingly. Right click the name of the project and go into properties. Under *Configuration Properties* > *General*, ensure that *Configuration Type* is set to *Dynamic Library (.dll)*, and under *Configuration Properties* > *Advanced*, under *C++/CLI Properties* set *Common Language Runtime Support* to *Common Language Runtime Support (/clr)*.

#### The C++ Side

To let the DLL know that you are writing a function for an external platform, functions need to be wrapped in a syntax. Consider a function prototype defined in a header file, consider a function

```
void function(typename2 arg1, typename2 arg2);
```

Instead we will write it as:

```
extern "C" {
    __declspec(dllexport) void function(typename1 arg1, typename2 arg2);
}
```

To create an interface between the library and C#, a common data structure must be defined, this data structure should contain only primitive types (built in types such as int, double, char, etc). I have not experimented with more complex types. Your mileage may vary. An example of a simple struct to use is:

```
struct Point {
    float x;
    float y;
    float z;

    Point(float x_, float y_, float z_) : x(x_), y(y_), z(z_) {}
};
```
Here I am choosing to use float values as the Unity package uses almost entirely floats as the real primitive. This is pretty straightforward.



#### The C# Side

Now we need to consider the Unity project. Essentially, we need to represent the header we would normally use with a shared library within the C# scripts, we also need to recreate the shared structs. The project must also include a *using* line near the beginning: `using System.Runtime.InteropServices;`

First we will recreate the function prototypes.

```
[DllImport("<DLL NAME>", CallingConvention = CallingConvention.Cdecl)]
    public static extern void function(typename1 arg1, typename2 arg2);
```
To match the shared struct, it needs to be wrapped in some C# syntax.

```
[StructLayout(LayoutKind.Sequential, CharSet = CharSet.Ansi)]
public struct PointStruct
{
    public float x;
    public float y;
    public float z;
}
```

Of course, the DLL itself must be copied into the project directory under Assets/Plugins.
